In [ ]:
## filt i/o fuction
def filein(datain):
    a = open(datain,'r')
    return [s.replace("\n","") for s in a]
def fileout(datain,path):
    a = open(path,"w")
    for i in datain:
        a.write(','.join(str(s) for s in i)+'\n')
    a.close()
    return a

## HLA allele matching function
def match_process(ori,gene):
    df = ori
    for i in [gene+".match",gene+".wrong",gene+".empty"]:
        df[0].append(i)
    for index,i in enumerate(df[1:]):
        match = 0
        wrong = 0
        empty = 0
        sm = i[0:1+1]
        ngs = i[2:3+1]
        if (sm.count("NA") == 1) | (ngs.count("NA") == 1):
            empty = 1
        elif (sm.count("NA") == 2) | (ngs.count("NA") ==2):
            empty = 2 
        life = empty
        for a in sm:
            if match + wrong + empty == 2:
                break
            if a in ngs:
                match = match + 1
                ngs.remove(a)
            elif (a not in ngs) & (life == 0):
                wrong = wrong +  1
            elif (a not in ngs) & (life == 1):
                life = life -  1
        df[index+1].append(match)
        df[index+1].append(wrong)
        df[index+1].append(empty)
    return df

## header function
def gene_df(ori,a1,a2,a3,a4):
    outdf = []
    for i in ori:
        x = []
        x.append(i[int(a1)])
        x.append(i[int(a2)])
        x.append(i[int(a3)])
        x.append(i[int(a4)])
        outdf.append(x)
    return outdf 
    
#### digit
digit = input("Enter digit : ")

genes = ["A","B","C","DRB1","DPA1","DPB1","DQA1","DQB1"]
in_df = filein(wdir + "MERGE.impResult.hlatyping.all.gene.%sdigit.csv"%digit)

index = in_df[0].split(",")

out = []
in_df = [s.split(",") for s in in_df]

for gene in genes:
    IMP_allele1 = index.index("IMP_%s.1"%gene)
    IMP_allele2 = index.index("IMP_%s.2"%gene)
    NGS_allele1 = index.index("NGS_%s.1"%gene)
    NGS_allele2 = index.index("NGS_%s.2"%gene)
    pre_df = gene_df(in_df,IMP_allele1,IMP_allele2,NGS_allele1,NGS_allele2)
    pre_df = match_process(pre_df,gene)
    for i,j in enumerate(pre_df):
        for k in j:
            out[i].append(k)

fileout(out,wdir + "compare.IMPvsNGS.all.gene.%sdigit.csv"%(digit))